#### Wednesday, April 3, 2024

Let's see if this still works, shall we ... 

Currently using LMStudio 0.2.18 

#### Wednesday, January 24, 2024

Code sample from ...

https://github.com/microsoft/autogen/discussions/237

Nice! Got this to work. "api_base" fails, but "base_url" works! Great!

Since we are using LMStudio, let's see if this code runs ...

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

# 3.0s Yup! This works! Nice!

ChatCompletionMessage(content=" In verse and rhythm, I'm here to entertain,\nI'm your AI, with a brain so serene.\nThrough words that flow, both near and far,\nIn the language of rhyme, I'll take you there.\nWelcome to my world, where stories unfold,\nA place of wonder, where magic is told.\nI'm here to inspire, to bring a smile,\nWith every line, I'll make your heart worthwhile.", role='assistant', function_call=None, tool_calls=None)


In [2]:
!nvidia-smi

Wed Apr  3 17:24:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        Off | 00000000:01:00.0  On |                  N/A |
|  0%   57C    P0              N/A /  70W |   1376MiB /  2048MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

So this next cell fails, and I am gonna guess it has to do with a depricated api ... so not gonna try to fix it, and yeah, I did pip uninstall autogenstudio then pip install autogenstudio, and it still fails ... meh for now. Gonna try a newer code sample ...

In [3]:
from autogen import AssistantAgent, UserProxyAgent, oai, GroupChat, GroupChatManager


ImportError: cannot import name 'AssistantAgent' from 'autogen' (/home/rob/miniforge3/envs/autogen/lib/python3.11/site-packages/autogen/__init__.py)

In [2]:
config_list=[
    {
        "model": "LMStudio Local Model",
        "base_url": "http://localhost:1234/v1",
        "api_type": "open_ai",
        "api_key": "NULL", # required, you don´t need to change this
        }
]

In [3]:
llm_config={
    "config_list": config_list,
}

In [4]:
coder = AssistantAgent(
    name="Coder",
    system_message="You write the code to solve the required Task. Implement improvements provided by the critic.",
    llm_config=llm_config,
)

In [5]:
critic = AssistantAgent(
    name="critic",
    system_message="You will make sure that the code provided by coder is bug free and all requirements are met. also provide coder with ideas for more features that make the project better. do not provide any code.",
    llm_config=llm_config
)

In [6]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    system_message="A human admin wo will give the idea and run the code provided by Coder.",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

In [7]:
groupchat = GroupChat(agents=[user_proxy, coder, critic], messages=[], max_round=12)

In [8]:
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

The next cell is what initiates a call to the server.

In [9]:
user_proxy.initiate_chat(manager, message="Write a calculator app using tkinter for the gui. the GUI Layout should resemble the layout of a casio calculator.", clear_history=False, config_list=config_list)

user_proxy (to chat_manager):

Write a calculator app using tkinter for the gui. the GUI Layout should resemble the layout of a casio calculator.

--------------------------------------------------------------------------------


GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:


```python
import tkinter as tk
from tkinter import messagebox

class Calculator(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        self.result = tk.Entry(self, font=("Arial", 18), bd=20)
        self.result.grid(row=0, column=0, columnspan=4)

        buttons = [
            ("7", 1, 0),
            ("8", 1, 1),
            ("9", 1, 2),
            ("/", 1, 3),
            ("4", 2, 0),
            ("5", 2, 1),
            ("6", 2, 2),
            ("*", 2, 3),
            ("1", 3, 0),
            ("2", 3, 1),
            ("3", 3, 2),
            ("-", 3, 3),
            ("0", 4, 0),
            (".", 4, 1),
            ("+", 4, 2),
            ("=", 4, 3),
        ]

        for (text, row, col) in buttons:
            button = tk.Button(self, t

Coder (to chat_manager):

First, I'll provide you with a simple Tkinter calculator app layout that resembles a Casio calculator. Then, I'll suggest some improvements based on your feedback.

```python
import tkinter as tk
from tkinter import messagebox

class Calculator(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Calculator")
        self.geometry("300x450")

        self.result_var = tk.StringVar()

        self.create_widgets()

    def create_widgets(self):
        entry = tk.Entry(self, textvariable=self.result_var, font=("Arial", 24), bd=10)
        entry.grid(row=0, column=0, columnspan=4)

        buttons = [
            ("7", 1, 0), ("8", 1, 1), ("9", 1, 2), ("/", 1, 3),
            ("4", 2, 0), ("5", 2, 1), ("6", 2, 2), ("*", 2, 3),
            ("1", 3, 0), ("2", 3, 1), ("3", 3, 2), ("-", 3, 3),
            ("0", 4, 0), (".", 4, 1), ("+", 4, 2), ("=", 4, 3),
        ]

        for (text, row, col) in buttons:
            button = tk.Button(s

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:



user_proxy (to chat_manager):

asf

--------------------------------------------------------------------------------


...

Coder (to chat_manager):

It seems like your message got cut off before providing any instruction. However, if you're asking for suggestions on how to make the calculator more user-friendly, here are some ideas:

1. Add support for decimal points: Currently, users cannot enter decimal numbers directly into the calculator. You can modify the `on_button_click` function to handle the "." button and update the expression accordingly.
2. Add a backspace button: Users may make mistakes while entering long expressions or equations. A backspace button would allow them to correct errors easily.
3. Use a theme or customize the appearance of buttons: You can use Tkinter's `ttk` module and its `Style` class to define a custom theme for your calculator. Alternatively, you can add images or icons to the buttons to make them more visually appealing and easier to identify.
4. Use Python's built-in `eval()` function with caution: While `eval()` is convenient for evaluating expressions entered by the us

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:



critic (to chat_manager):

You've provided an excellent example of creating a simple calculator GUI in Tkinter! I appreciate your attention to detail and organization, especially with defining the buttons using lists. Your suggestions for improvements are on point as well. Adding images, custom themes, and support for complex equations will definitely enhance the user experience and functionality of this calculator. Keep up the good work!

Regarding the backspace feature, you've correctly implemented it by deleting the last character from the `StringVar` when the backspace button is clicked. This ensures that the display reflects the updated value after each backspace press.

As for the decimal point, your current implementation only adds a new decimal point if there isn't already one in the input string. To allow multiple decimal points, you may want to modify the `on_button_click` function as follows:
```python
def on_button_click(self, button_text):
    if button_text == "C":
      

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:



user_proxy (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------
